Spyros has gone through the plate annotations and re-annotated the cells using the controlled vocabulary of the [Cell Ontology](http://cellontology.org)

In [1]:
import pandas as pd
import os
import numpy as np

from maca import clean_annotation, clean_labels

metadata_folder = os.path.join('..', 'metadata' )

summary_folder = os.path.join(metadata_folder, 'summary')

annotation_folder = os.path.join(metadata_folder, 'manual_annotations')


%load_ext autoreload
%autoreload 2

In [2]:
ontology_renamer = {"Class": 'cell_ontology_class', 'Term IRI': "cell_ontology_term_iri"}
ontology_renamer

{'Class': 'cell_ontology_class', 'Term IRI': 'cell_ontology_term_iri'}

In [3]:


csv = os.path.join(annotation_folder, 'maca_3month_onotology_annotations_plates.csv')
plates_ontology = pd.read_csv(csv)
plates_ontology = plates_ontology.fillna('.')
plates_ontology['annotation_cleaned'] = plates_ontology['annotation_cleaned'].map(lambda x: x.split('_cells')[0]
        if not x in ('t_cells', 'b_cells') else x)

plates_ontology['annotation_cleaned'] = plates_ontology['annotation_cleaned'].replace(
    'macrophages/dendritic','myeloid')
plates_ontology['annotation_cleaned'] = plates_ontology['annotation_cleaned'].replace(
    'stromal_mesenchymal_stem_cell','stromal_stem')


old_to_new_name = {'Mammary_Gland': "Mammary",
            "Brain_FACS_microglia": "Brain_(Microglia)",
            "Brain_FACS_neurons": "Brain_(Non-microglia)"}
plates_ontology['tissue'] = plates_ontology['tissue'].replace(old_to_new_name)
# cols_to_drop = [x for x in plates_with_ontology if x.startswith('Unnamed')]
# plates_with_ontology = plates_with_ontology.drop(cols_to_drop, axis=1)
print(plates_ontology.shape)
plates_ontology.head(20)

(211, 8)


,tissue,annotation_original,subannotation_original,annotation_cleaned,subannotation_cleaned,n_cells,Class,Term IRI
0,Aorta,adipocytes,.,adipocytes,.,54,epicardial adipocyte,http://purl.obolibrary.org/obo/CL_1000309
1,Aorta,heterogenous group of cells,adipocytes,adipocytes,.,39,epicardial adipocyte,http://purl.obolibrary.org/obo/CL_1000309
2,Aorta,endothelial cells,.,endothelial,.,131,endothelial cell,http://purl.obolibrary.org/obo/CL_0000115
3,Aorta,fibroblasts,.,fibroblasts,.,35,fibroblast,http://purl.obolibrary.org/obo/CL_0000057
4,Aorta,hematopoetic cells,.,hematopoietic,.,17,hematopoietic cell,http://purl.obolibrary.org/obo/CL_0000988
5,Aorta,heterogenous group of cells,smooth muscle cells,smooth_muscle,.,37,smooth muscle cell,http://purl.obolibrary.org/obo/CL_0000192
6,Aorta,heterogenous group of cells,erythroblasts and adipocytes,unknown,.,51,.,.
7,Bladder,Basal,.,basal,.,99,basal cell of urothelium,http://purl.obolibrary.org/obo/CL_1000486
8,Bladder,LuminalA1,.,luminal,a,191,bladder cell,http://purl.obolibrary.org/obo/CL_1001319
9,Bladder,LuminalA2,.,luminal,a,245,bladder cell,http://purl.obolibrary.org/obo/CL_1001319


In [4]:
plates_ontology.head().values

array([['Aorta', 'adipocytes', '.', 'adipocytes', '.', 54,
        'epicardial adipocyte', 'http://purl.obolibrary.org/obo/CL_1000309'],
       ['Aorta', 'heterogenous group of cells', 'adipocytes', 'adipocytes',
        '.', 39, 'epicardial adipocyte',
        'http://purl.obolibrary.org/obo/CL_1000309'],
       ['Aorta', 'endothelial cells', '.', 'endothelial', '.', 131,
        'endothelial cell', 'http://purl.obolibrary.org/obo/CL_0000115'],
       ['Aorta', 'fibroblasts', '.', 'fibroblasts', '.', 35, 'fibroblast',
        'http://purl.obolibrary.org/obo/CL_0000057'],
       ['Aorta', 'hematopoetic cells', '.', 'hematopoietic', '.', 17,
        'hematopoietic cell', 'http://purl.obolibrary.org/obo/CL_0000988']], dtype=object)

In [5]:
# plates_ontology.groupby(['tissue', 'annotation_cleaned','subannotation_cleaned', 'Class']).size().to_frame()

In [6]:
ontology_cols = ['tissue', 'annotation_cleaned', 'subannotation_cleaned', 'Class', 'Term IRI']

# plate_ontology_subset = plates_ontology[ontology_cols].replace('.', np.nan)
print(plates_ontology.shape)
plates_ontology = plates_ontology.drop_duplicates(subset=ontology_cols)
print(plates_ontology.shape)
plates_ontology.head()

(211, 8)
(155, 8)


,tissue,annotation_original,subannotation_original,annotation_cleaned,subannotation_cleaned,n_cells,Class,Term IRI
0,Aorta,adipocytes,.,adipocytes,.,54,epicardial adipocyte,http://purl.obolibrary.org/obo/CL_1000309
2,Aorta,endothelial cells,.,endothelial,.,131,endothelial cell,http://purl.obolibrary.org/obo/CL_0000115
3,Aorta,fibroblasts,.,fibroblasts,.,35,fibroblast,http://purl.obolibrary.org/obo/CL_0000057
4,Aorta,hematopoetic cells,.,hematopoietic,.,17,hematopoietic cell,http://purl.obolibrary.org/obo/CL_0000988
5,Aorta,heterogenous group of cells,smooth muscle cells,smooth_muscle,.,37,smooth muscle cell,http://purl.obolibrary.org/obo/CL_0000192


In [7]:
plate_ontology_subset = plates_ontology[ontology_cols]
plate_ontology_subset = plate_ontology_subset.set_index(['tissue', 'annotation_cleaned', 'subannotation_cleaned'])
plate_ontology_subset = plate_ontology_subset.sort_index()
plate_ontology_subset.index.names = ['tissue', 'annotation', 'subannotation']
print(plate_ontology_subset.shape)
plate_ontology_subset.head()

(155, 2)


Class  \
tissue annotation    subannotation                         
Aorta  adipocytes    .              epicardial adipocyte   
       endothelial   .                  endothelial cell   
       fibroblasts   .                        fibroblast   
       hematopoietic .                hematopoietic cell   
       smooth_muscle .                smooth muscle cell   

                                                                     Term IRI  
tissue annotation    subannotation                                             
Aorta  adipocytes    .              http://purl.obolibrary.org/obo/CL_1000309  
       endothelial   .              http://purl.obolibrary.org/obo/CL_0000115  
       fibroblasts   .              http://purl.obolibrary.org/obo/CL_0000057  
       hematopoietic .              http://purl.obolibrary.org/obo/CL_0000988  
       smooth_muscle .              http://purl.obolibrary.org/obo/CL_0000192

### Read annotation

In [8]:
csv = os.path.join(metadata_folder, 'maca_3month_annotations_plates.csv')
plate_annotation = pd.read_csv(csv, index_col=0)
plate_annotation = plate_annotation.fillna('.')
print(plate_annotation.shape)
plate_annotation.head()

(42192, 5)


,annotation,plate.barcode,subannotation,tissue,annotation_subannotation
A21.MAA000594.3_8_M.1.1,fibroblasts,MAA000594,.,Aorta,fibroblasts
F8.MAA000594.3_8_M.1.1,unknown,MAA000594,.,Aorta,unknown
H11.MAA000594.3_8_M.1.1,unknown,MAA000594,.,Aorta,unknown
A22.MAA000594.3_8_M.1.1,unknown,MAA000594,.,Aorta,unknown
H12.MAA000594.3_8_M.1.1,adipocytes,MAA000594,.,Aorta,adipocytes


In [9]:
left_on = ['tissue', 'annotation', 'subannotation']


In [10]:
plate_annotation.query('tissue == "Thymus"').groupby(left_on).size().to_frame()

0
tissue annotation   subannotation                                      
Thymus stromal_stem .                                                33
       t_cells      double_negative1                                 32
                    double_negative4                                 92
                    double_positive                                  44
                    double_positive_favoring_cd8+                   637
                    double_positive_rapidly_dividing_favoring_cd8+  103
                    immature_single_positive_cd4+                   280
                    immature_single_positive_cd8+                    62

In [11]:
plate_ontology_subset.loc['Thymus', :]

Class  \
annotation   subannotation                                                           
stromal_stem .                                               mesenchymal stem cell   
t_cells      double_negative1                                               T cell   
             double_negative4                                               T cell   
             double_positive                                                T cell   
             double_positive_favoring_cd8+                                  T cell   
             double_positive_rapidly_dividing_favoring_cd8+                 T cell   
             immature_single_positive_cd4+                                  T cell   
             immature_single_positive_cd8+                                  T cell   

                                                                                              Term IRI  
annotation   subannotation                                                                              
stromal_stem .                                               http://purl.obolibrary.org/obo/CL_0000134  
t_cells      double_negative1                                http://purl.obolibrary.org/obo/CL_0000084  
             double_negative4                                http://purl.obolibrary.org/obo/CL_0000084  
             double_positive                                 http://purl.obolibrary.org/obo/CL_0000084  
             double_positive_favoring_cd8+                   http://purl.obolibrary.org/obo/CL_0000084  
             double_positive_rapidly_dividing_favoring_cd8+  http://purl.obolibrary.org/obo/CL_0000084  
             immature_single_positive_cd4+                   http://purl.obolibrary.org/obo/CL_0000084  
             immature_single_positive_cd8+                   http://purl.obolibrary.org/obo/CL_0000084

In [12]:
pd.options.display.max_rows = 100

In [13]:
# right_on = ['tissue', 'annotation_cleaned', 'subannotation_cleaned']
plate_annotation_with_ontology = plate_annotation.join(
    plate_ontology_subset, on=left_on)
# plate_annotation_with_ontology = plate_annotation_with_ontology.drop_duplicates()
print(plate_annotation_with_ontology.shape)
plate_annotation_with_ontology.fillna('NA').groupby(left_on + ['Class']).size().to_frame()

(42192, 7)


0
tissue                annotation                subannotation                                  Class                                                   
Aorta                 adipocytes                .                                              epicardial adipocyte                                  93
                      endothelial               .                                              endothelial cell                                     131
                      fibroblasts               .                                              fibroblast                                            35
                      hematopoietic             .                                              hematopoietic cell                                    17
                      smooth_muscle             .                                              smooth muscle cell                                    37
                      unknown                   .                                              .                                                     51
Bladder               basal                     .                                              basal cell of urothelium                              99
                      luminal                   a                                              bladder cell                                         436
                                                b                                              bladder cell                                          96
                      mesenchymal               a                                              mesenchymal cell                                     335
                                                b                                              mesenchymal cell                                     321
Brain_(Microglia)     macrophages               .                                              macrophage                                            36
                      microglia                 .                                              microglial cell                                     4329
Brain_(Non-microglia) astrocytes                berg_glia                                      NA                                                    30
                                                cerebral_astrocytes                            astrocyte of the cerebral cortex                     403
                                                myelinating_oligodendrocytes                   NA                                                    11
                      endothelial               vascular                                       endothelial cell                                     625
                                                vsmcs                                          NA                                                    14
                      neurons                   doublet                                        NA                                                     2
                                                excitatory                                     neuron                                                73
                                                inhibitory                                     neuron                                                98
                                                undetermined                                   neuron                                                23
                      npcs                      .                                              NA                                                    36
                      oligodendrocytes          .                                              oligodendrocyte                                     1175
                      opcs                      .                                              NA                                                   202
                      pericytes               

In [14]:
plate_annotation_with_ontology.loc[plate_annotation_with_ontology['Class'].isnull()].drop_duplicates()

,annotation,plate.barcode,subannotation,tissue,annotation_subannotation,Class,Term IRI
E19.MAA000538.3_8_M.1.1,astrocytes,MAA000538,berg_glia,Brain_(Non-microglia),astrocytes: berg_glia,NaN,NaN
G7.MAA000550.3_10_M.1.1,opcs,MAA000550,.,Brain_(Non-microglia),opcs,NaN,NaN
I11.MAA000553.3_10_M.1.1,opcs,MAA000553,.,Brain_(Non-microglia),opcs,NaN,NaN
L18.MAA000560.3_10_M.1.1,opcs,MAA000560,.,Brain_(Non-microglia),opcs,NaN,NaN
O15.MAA000560.3_10_M.1.1,astrocytes,MAA000560,myelinating_oligodendrocytes,Brain_(Non-microglia),astrocytes: myelinating_oligodendrocytes,NaN,NaN
B4.MAA000560.3_10_M.1.1,endothelial,MAA000560,vsmcs,Brain_(Non-microglia),endothelial: vsmcs,NaN,NaN
B21.MAA000560.3_10_M.1.1,npcs,MAA000560,.,Brain_(Non-microglia),npcs,NaN,NaN
J15.MAA000561.3_10_M.1.1,neurons,MAA000561,doublet,Brain_(Non-microglia),neurons: doublet,NaN,NaN
J19.MAA000561.3_10_M.1.1,npcs,MAA000561,.,Brain_(Non-microglia),npcs,NaN,NaN
A11.MAA000561.3_10_M.1.1,opcs,MAA000561,.,Brain_(Non-microglia),opcs,NaN,NaN


In [15]:
plate_annotation_with_ontology.head()

,annotation,plate.barcode,subannotation,tissue,annotation_subannotation,Class,Term IRI
A21.MAA000594.3_8_M.1.1,fibroblasts,MAA000594,.,Aorta,fibroblasts,fibroblast,http://purl.obolibrary.org/obo/CL_0000057
F8.MAA000594.3_8_M.1.1,unknown,MAA000594,.,Aorta,unknown,.,.
H11.MAA000594.3_8_M.1.1,unknown,MAA000594,.,Aorta,unknown,.,.
A22.MAA000594.3_8_M.1.1,unknown,MAA000594,.,Aorta,unknown,.,.
H12.MAA000594.3_8_M.1.1,adipocytes,MAA000594,.,Aorta,adipocytes,epicardial adipocyte,http://purl.obolibrary.org/obo/CL_1000309


### Add cell ontology ID

In [16]:
plate_annotation_with_ontology['cell_ontology_id'] = 'CL:' + plate_annotation_with_ontology['Term IRI'].str.split('_').str[-1]
plate_annotation_with_ontology.head()

,annotation,plate.barcode,subannotation,tissue,annotation_subannotation,Class,Term IRI,cell_ontology_id
A21.MAA000594.3_8_M.1.1,fibroblasts,MAA000594,.,Aorta,fibroblasts,fibroblast,http://purl.obolibrary.org/obo/CL_0000057,CL:0000057
F8.MAA000594.3_8_M.1.1,unknown,MAA000594,.,Aorta,unknown,.,.,CL:.
H11.MAA000594.3_8_M.1.1,unknown,MAA000594,.,Aorta,unknown,.,.,CL:.
A22.MAA000594.3_8_M.1.1,unknown,MAA000594,.,Aorta,unknown,.,.,CL:.
H12.MAA000594.3_8_M.1.1,adipocytes,MAA000594,.,Aorta,adipocytes,epicardial adipocyte,http://purl.obolibrary.org/obo/CL_1000309,CL:1000309


### Unify NAs

In [17]:
plate_annotation_with_ontology = plate_annotation_with_ontology.replace('.', np.nan).replace('NA', np.nan)
print(plate_annotation_with_ontology.isnull().sum().sum())
plate_annotation_with_ontology['Class'] = plate_annotation_with_ontology['Class'].fillna('unknown')
print(plate_annotation_with_ontology.isnull().sum().sum())
plate_annotation_with_ontology.head()

27932
27410


,annotation,plate.barcode,subannotation,tissue,annotation_subannotation,Class,Term IRI,cell_ontology_id
A21.MAA000594.3_8_M.1.1,fibroblasts,MAA000594,NaN,Aorta,fibroblasts,fibroblast,http://purl.obolibrary.org/obo/CL_0000057,CL:0000057
F8.MAA000594.3_8_M.1.1,unknown,MAA000594,NaN,Aorta,unknown,unknown,NaN,CL:.
H11.MAA000594.3_8_M.1.1,unknown,MAA000594,NaN,Aorta,unknown,unknown,NaN,CL:.
A22.MAA000594.3_8_M.1.1,unknown,MAA000594,NaN,Aorta,unknown,unknown,NaN,CL:.
H12.MAA000594.3_8_M.1.1,adipocytes,MAA000594,NaN,Aorta,adipocytes,epicardial adipocyte,http://purl.obolibrary.org/obo/CL_1000309,CL:1000309


### Make all columns lowercase with underscores

In [18]:
plate_annotation_with_ontology = plate_annotation_with_ontology.rename(columns=ontology_renamer)
plate_annotation_with_ontology.head()

,annotation,plate.barcode,subannotation,tissue,annotation_subannotation,cell_ontology_class,cell_ontology_term_iri,cell_ontology_id
A21.MAA000594.3_8_M.1.1,fibroblasts,MAA000594,NaN,Aorta,fibroblasts,fibroblast,http://purl.obolibrary.org/obo/CL_0000057,CL:0000057
F8.MAA000594.3_8_M.1.1,unknown,MAA000594,NaN,Aorta,unknown,unknown,NaN,CL:.
H11.MAA000594.3_8_M.1.1,unknown,MAA000594,NaN,Aorta,unknown,unknown,NaN,CL:.
A22.MAA000594.3_8_M.1.1,unknown,MAA000594,NaN,Aorta,unknown,unknown,NaN,CL:.
H12.MAA000594.3_8_M.1.1,adipocytes,MAA000594,NaN,Aorta,adipocytes,epicardial adipocyte,http://purl.obolibrary.org/obo/CL_1000309,CL:1000309


### Write CSV

In [19]:
csv = os.path.join(metadata_folder, 'maca_3month_annotations_plates_ontology.csv')
plate_annotation_with_ontology.to_csv(csv)

### Write CSV of cell ontology counts

In [39]:
cols = ['tissue', 'cell_ontology_class', 'cell_ontology_id', 'cell_ontology_term_iri']
summary = plate_annotation_with_ontology.groupby(cols).size().reset_index()
summary = summary.rename(columns={0: 'n_cells'})

csv = os.path.join(summary_folder, 'maca_3month_cell_ontology_counts_plates.csv')
summary.to_csv(csv, index=False)

## Replace 10x tooo

### Read 10x ontology

In [20]:
csv = os.path.join(annotation_folder, 'maca_3month_onotology_annotations_10x.csv')
tenx_ontology = pd.read_csv(csv)
tenx_ontology = tenx_ontology.fillna('.')
tenx_ontology['annotation_cleaned'] = tenx_ontology['annotation_cleaned'].map(lambda x: x.split('_cells')[0]
        if not x in ('t_cells', 'b_cells') else x)

tenx_ontology['annotation_cleaned'] = tenx_ontology['annotation_cleaned'].replace(
    'macrophages/dendritic','myeloid')
tenx_ontology['annotation_cleaned'] = tenx_ontology['annotation_cleaned'].replace(
    'stromal_mesenchymal_stem_cell','stromal_stem')


# old_to_new_name = {'Mammary_Gland': "Mammary",
#             "Brain_FACS_microglia": "Brain_Microglia",
#             "Brain_FACS_neurons": "Brain_Neurons"}
tenx_ontology['tissue'] = tenx_ontology['tissue'].replace(old_to_new_name)
# cols_to_drop = [x for x in tenx_with_ontology if x.startswith('Unnamed')]
# tenx_with_ontology = tenx_with_ontology.drop(cols_to_drop, axis=1)
print(tenx_ontology.shape)
tenx_ontology.head(20)

(125, 8)


,tissue,annotation_original,subannotation_original,annotation_cleaned,subannotation_cleaned,n_cells,Class,Term IRI
0,Bladder,Basal,.,basal,.,266,basal cell of urothelium,http://purl.obolibrary.org/obo/CL_1000486
1,Bladder,Endothelial cells,.,endothelial,.,68,endothelial cell,http://purl.obolibrary.org/obo/CL_0000115
2,Bladder,Immune cells,.,immune,.,57,leukocyte,http://purl.obolibrary.org/obo/CL_0000738
3,Bladder,LuminalA1,.,luminal,a,313,bladder cell,http://purl.obolibrary.org/obo/CL_1001319
4,Bladder,LuminalA2,.,luminal,a,219,bladder cell,http://purl.obolibrary.org/obo/CL_1001319
5,Bladder,LuminalB,.,luminal,b,391,bladder cell,http://purl.obolibrary.org/obo/CL_1001319
6,Bladder,MesenchymalA1,.,mesenchymal,a,169,mesenchymal cell,http://purl.obolibrary.org/obo/CL_0008019
7,Bladder,MesenchymalA2,.,mesenchymal,a,501,mesenchymal cell,http://purl.obolibrary.org/obo/CL_0008019
8,Bladder,MesenchymalB1,.,mesenchymal,b,187,mesenchymal cell,http://purl.obolibrary.org/obo/CL_0008019
9,Bladder,MesenchymalB2,.,mesenchymal,b,329,mesenchymal cell,http://purl.obolibrary.org/obo/CL_0008019


In [21]:
# plate_ontology_subset = tenx_ontology[ontology_cols].replace('.', np.nan)
print(tenx_ontology.shape)
tenx_ontology = tenx_ontology.drop_duplicates(subset=ontology_cols)
print(tenx_ontology.shape)
tenx_ontology.head()

(125, 8)
(110, 8)


,tissue,annotation_original,subannotation_original,annotation_cleaned,subannotation_cleaned,n_cells,Class,Term IRI
0,Bladder,Basal,.,basal,.,266,basal cell of urothelium,http://purl.obolibrary.org/obo/CL_1000486
1,Bladder,Endothelial cells,.,endothelial,.,68,endothelial cell,http://purl.obolibrary.org/obo/CL_0000115
2,Bladder,Immune cells,.,immune,.,57,leukocyte,http://purl.obolibrary.org/obo/CL_0000738
3,Bladder,LuminalA1,.,luminal,a,313,bladder cell,http://purl.obolibrary.org/obo/CL_1001319
5,Bladder,LuminalB,.,luminal,b,391,bladder cell,http://purl.obolibrary.org/obo/CL_1001319


In [22]:
tenx_ontology_subset = tenx_ontology[ontology_cols]
tenx_ontology_subset = tenx_ontology_subset.set_index(['tissue', 'annotation_cleaned', 'subannotation_cleaned'])
tenx_ontology_subset = tenx_ontology_subset.sort_index()
tenx_ontology_subset.index.names = ['tissue', 'annotation', 'subannotation']
print(tenx_ontology_subset.shape)
tenx_ontology_subset.head()

(110, 2)


Class  \
tissue  annotation  subannotation                             
Bladder basal       .              basal cell of urothelium   
        endothelial .                      endothelial cell   
        immune      .                             leukocyte   
        luminal     a                          bladder cell   
                    b                          bladder cell   

                                                                    Term IRI  
tissue  annotation  subannotation                                             
Bladder basal       .              http://purl.obolibrary.org/obo/CL_1000486  
        endothelial .              http://purl.obolibrary.org/obo/CL_0000115  
        immune      .              http://purl.obolibrary.org/obo/CL_0000738  
        luminal     a              http://purl.obolibrary.org/obo/CL_1001319  
                    b              http://purl.obolibrary.org/obo/CL_1001319

### Read 10x annotation

In [23]:
csv = os.path.join(metadata_folder, 'maca_3month_annotations_10x.csv')
tenx_annotation = pd.read_csv(csv, index_col=0)
tenx_annotation = tenx_annotation.fillna('.')
print(tenx_annotation.shape)
tenx_annotation.head()

(54837, 5)


,annotation,subannotation,tissue,annotation_subannotation,tenx_channel
10X_P4_3_AAAGTAGAGATGCCAG,mesenchymal,b,Bladder,mesenchymal: b,10X_P4_3
10X_P4_3_AACCGCGTCCAACCAA,mesenchymal,a,Bladder,mesenchymal: a,10X_P4_3
10X_P4_3_AACTCCCGTCGGGTCT,mesenchymal,a,Bladder,mesenchymal: a,10X_P4_3
10X_P4_3_AACTCTTAGTTGCAGG,luminal,a,Bladder,luminal: a,10X_P4_3
10X_P4_3_AACTCTTTCATAACCG,mesenchymal,a,Bladder,mesenchymal: a,10X_P4_3


### Join the ontology and annotations

In [24]:
# right_on = ['tissue', 'annotation_cleaned', 'subannotation_cleaned']
tenx_annotation_with_ontology = tenx_annotation.join(
    tenx_ontology_subset, on=left_on)
# tenx_annotation_with_ontology = tenx_annotation_with_ontology.drop_duplicates()
print(tenx_annotation_with_ontology.shape)
tenx_annotation_with_ontology.fillna('NA').groupby(left_on + ['Class']).size().to_frame()

(54837, 7)


0
tissue  annotation            subannotation                         Class                                    
Bladder basal                 .                                     basal cell of urothelium              266
        endothelial           .                                     endothelial cell                       68
        immune                .                                     leukocyte                              57
        luminal               a                                     bladder cell                          532
                              b                                     bladder cell                          391
        mesenchymal           a                                     mesenchymal cell                      670
                              b                                     mesenchymal cell                      516
Heart   cardiomyocytes        .                                     cardiac muscle cell                    83
        endothelial           coronary_vascular                     endothelial cell                      178
                              endocardial                           endocardial cell                       65
        erythrocytes          .                                     erythrocyte                            55
        fibroblasts           .                                     fibroblast                            222
        smooth_muscle         .                                     smooth muscle cell                     21
Kidney  collecting_duct       .                                     kidney collecting duct cell            72
        endothelial           .                                     endothelial cell                       45
        fenestrated_capillary .                                     fenestrated cell                      345
        fibroblasts           .                                     fibroblast                            113
        immune                .                                     leukocyte                              54
        macrophages           .                                     macrophage                            181
        smooth_muscle         .                                     smooth muscle cell                     50
        tubule                connecting                            kidney tubule cell                    245
                              proximal                              kidney tubule cell                    851
                              proximal_ascending                    kidney tubule cell                    352
                              thick_ascending                       kidney tubule cell                    473
Liver   endothelial           .                                     endothelial cell                       20
        hepatocytes           midlobular                            hepatocyte                            264
                              pericentral                           hepatocyte                            373
                              periportal                            hepatocyte                            369
Lung    b_cells               .                                     B cell                                205
        ciliated              .                                     ciliated cell                          41
        club                  .                                     Clara cell                              5
        dendritic             .                                     dendritic cell                        225
        endothelial           .                                     endothelial cell                      425
        epithelial            alveolar_type_ii                      type II pneumocyte                     89
        immune                .                                     leukocyte                             151
        macrophages           alveolar

In [25]:
tenx_annotation_with_ontology.loc[tenx_annotation_with_ontology['Class'].isnull()].drop_duplicates()

,annotation,subannotation,tissue,annotation_subannotation,tenx_channel,Class,Term IRI


In [26]:
tenx_annotation_with_ontology.loc[tenx_annotation_with_ontology['annotation'] == 'unknown'].drop_duplicates()

,annotation,subannotation,tissue,annotation_subannotation,tenx_channel,Class,Term IRI
10X_P7_8_ACTTGTTCATATGGTC,unknown,.,Lung,unknown,10X_P7_8,.,.
10X_P7_9_AGGCCGTGTGCGATAG,unknown,.,Lung,unknown,10X_P7_9,.,.
10X_P8_12_AGTTGGTGTAGAGTGC,unknown,.,Lung,unknown,10X_P8_12,.,.
10X_P8_13_ATTATCCAGACTTGAA,unknown,.,Lung,unknown,10X_P8_13,.,.
10X_P7_14_AAACCTGGTGCCTGGT,unknown,.,Muscle,unknown,10X_P7_14,.,.
10X_P7_15_AAATGCCTCCATGAAC,unknown,.,Muscle,unknown,10X_P7_15,.,.
10X_P4_0_AAGGCAGGTGGTTTCA,unknown,.,Tongue,unknown,10X_P4_0,.,.
10X_P4_1_AATCCAGTCTGTCAAG,unknown,.,Tongue,unknown,10X_P4_1,.,.
10X_P7_10_ACGGGCTAGGGAAACA,unknown,.,Tongue,unknown,10X_P7_10,.,.


### Add cell ontology ID

In [27]:
tenx_annotation_with_ontology['cell_ontology_id'] = 'CL:' + tenx_annotation_with_ontology['Term IRI'].str.split('_').str[-1]
tenx_annotation_with_ontology.head()

,annotation,subannotation,tissue,annotation_subannotation,tenx_channel,Class,Term IRI,cell_ontology_id
10X_P4_3_AAAGTAGAGATGCCAG,mesenchymal,b,Bladder,mesenchymal: b,10X_P4_3,mesenchymal cell,http://purl.obolibrary.org/obo/CL_0008019,CL:0008019
10X_P4_3_AACCGCGTCCAACCAA,mesenchymal,a,Bladder,mesenchymal: a,10X_P4_3,mesenchymal cell,http://purl.obolibrary.org/obo/CL_0008019,CL:0008019
10X_P4_3_AACTCCCGTCGGGTCT,mesenchymal,a,Bladder,mesenchymal: a,10X_P4_3,mesenchymal cell,http://purl.obolibrary.org/obo/CL_0008019,CL:0008019
10X_P4_3_AACTCTTAGTTGCAGG,luminal,a,Bladder,luminal: a,10X_P4_3,bladder cell,http://purl.obolibrary.org/obo/CL_1001319,CL:1001319
10X_P4_3_AACTCTTTCATAACCG,mesenchymal,a,Bladder,mesenchymal: a,10X_P4_3,mesenchymal cell,http://purl.obolibrary.org/obo/CL_0008019,CL:0008019


### Unify NAs

In [28]:
tenx_annotation_with_ontology.isnull().sum().sum()

0

In [29]:
tenx_annotation_with_ontology = tenx_annotation_with_ontology.replace('.', np.nan)
print(tenx_annotation_with_ontology.isnull().sum().sum())
tenx_annotation_with_ontology['Class'] = tenx_annotation_with_ontology['Class'].fillna('unknown')
tenx_annotation_with_ontology.isnull().sum().sum()

29397


28999

### Rename ontology columns

In [30]:
tenx_annotation_with_ontology = tenx_annotation_with_ontology.rename(columns=ontology_renamer)
tenx_annotation_with_ontology.head()

,annotation,subannotation,tissue,annotation_subannotation,tenx_channel,cell_ontology_class,cell_ontology_term_iri,cell_ontology_id
10X_P4_3_AAAGTAGAGATGCCAG,mesenchymal,b,Bladder,mesenchymal: b,10X_P4_3,mesenchymal cell,http://purl.obolibrary.org/obo/CL_0008019,CL:0008019
10X_P4_3_AACCGCGTCCAACCAA,mesenchymal,a,Bladder,mesenchymal: a,10X_P4_3,mesenchymal cell,http://purl.obolibrary.org/obo/CL_0008019,CL:0008019
10X_P4_3_AACTCCCGTCGGGTCT,mesenchymal,a,Bladder,mesenchymal: a,10X_P4_3,mesenchymal cell,http://purl.obolibrary.org/obo/CL_0008019,CL:0008019
10X_P4_3_AACTCTTAGTTGCAGG,luminal,a,Bladder,luminal: a,10X_P4_3,bladder cell,http://purl.obolibrary.org/obo/CL_1001319,CL:1001319
10X_P4_3_AACTCTTTCATAACCG,mesenchymal,a,Bladder,mesenchymal: a,10X_P4_3,mesenchymal cell,http://purl.obolibrary.org/obo/CL_0008019,CL:0008019


### Write CSVs

In [31]:
csv = os.path.join(metadata_folder, 'maca_3month_annotations_10x_ontology.csv')
tenx_annotation_with_ontology.to_csv(csv)

### write cell ontology counts

In [38]:
cols = ['tissue', 'cell_ontology_class', 'cell_ontology_id', 'cell_ontology_term_iri']
summary = tenx_annotation_with_ontology.groupby(cols).size().reset_index()
summary = summary.rename(columns={0: 'n_cells'})

csv = os.path.join(summary_folder, 'maca_3month_cell_ontology_counts_10x.csv')
summary.to_csv(csv, index=False)

